Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table

In [14]:
# подгружаем .env
load_dotenv()

True

In [15]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [16]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [17]:
# Укажем таблицу для удаления
TABLE_NAME = 'alt_wb_statistic_dag'

In [18]:
# Функция удаления таблицы с помощью sqlalchemy
def delete_table(table_name, conn):
    metadata = MetaData(bind=conn)
    table = Table(table_name, metadata, autoload=True)
    table.drop(conn)

In [19]:
# Удалим таблицу
delete_table(TABLE_NAME, dst_conn)

NoSuchTableError: alt_wb_statistic_dag

In [21]:
# Проверим, что таблицы больше нет
pd.read_sql(f'select * from {TABLE_NAME}', dst_conn)

,id,project_id_alt,countryname_off,countryname,countrycode,sector1,year,year_close,totalamt,vvp,population,electricity,rural,target
0,1,P163962,Democratic Republic of the Congo,None,COD,None,2018,2023,200,NaN,NaN,NaN,NaN,1
1,2,P167672,People's Republic of Bangladesh,None,BGD,None,2018,,58,NaN,NaN,NaN,NaN,0
2,3,P158768,Islamic Republic of Afghanistan,None,AFG,None,2018,2023,20,NaN,NaN,NaN,NaN,1
3,4,P161364,Federal Republic of Nigeria,None,NGA,Social Protection,2018,2023,100,NaN,NaN,NaN,NaN,1
4,5,P161483,Republic of Tunisia,None,TUN,None,2018,2019,500,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,5996,P100154,Federative Republic of Brazil,Brazil,BRA,Water Supply,2006,,50,1.107640e+12,189012412.0,97.594315,16.857,0
5996,5997,P100327,Republic of Indonesia,Indonesia,IDN,Other Public Administration,2006,2007,600,3.645705e+11,229838202.0,90.62,53.268,1
5997,5998,P101124,Oriental Republic of Uruguay,Uruguay,URY,Central Government (Central Agencies),2006,,6,1.957946e+10,3331043.0,98.505424,6.447,0
5998,5999,P101335,Republic of Peru,Peru,PER,Central Government (Central Agencies),2006,2007,200,8.864319e+10,27949944.0,80.15727,24.579,1


In [10]:
# Закроем соединения в конце работы
src_conn.dispose()
dst_conn.dispose()